# Generate full labelfile

Script to generate a json file which contains raw audio and spectrogram data for each call.

Requirements:

- A labelfile containing callID_new of all calls (generated with 00_1_generate_call_txts)
- A folder containig txt audio files of the calls (generated with 00_1_generate_call_txts)

Final results will be called "full_labelfile.pkl" and saved in current working directory.

In [1]:
import pandas as pd
import os
import librosa
import sys
import numpy as np
import pickle

In [2]:
print("Please type your path to the EAS server and press enter! \n For example: \n /Volumes    (on Mac) \n //10.126.19.90    (on Windows) \n /home/username/Documents/MPI-Server    (when mounted in Documents) \n")
SERVER=input()

Please type your path to the EAS server and press enter! 
 For example: 
 /Volumes    (on Mac) 
 //10.126.19.90    (on Windows) 
 /home/username/Documents/MPI-Server    (when mounted in Documents) 

/Volumes


In [3]:
if not os.path.exists(SERVER):
    print("Invalid server path: ", SERVER)
    exit()  
    
# If someone put a slash or backslash in last position
if SERVER[-1:]=="/" or SERVER[-1:]=="\n":
    SERVER = SERVER[:-1]

In [8]:
HOME = SERVER + os.path.join(os.path.sep, 'EAS_shared',
                                         'meerkat','working','processed',
                                         'acoustic', 'extract_calls')

# labelfile generated with 00_1_generate_call_txts
LABELFILE = os.path.join(os.path.sep, HOME,'labelfile.csv')

# location of audio txt files generated with 00_1_generate_call_txts
TXT_PATH = os.path.join(os.path.sep, HOME,'txts')

In [5]:
# Spectrogramming parameters
FFT_WIN = 0.03 # FFT_WIN*samplerate = length of fft/n_fft (number of audio frames that go in one fft)
FFT_HOP = FFT_WIN/8 # FFT_HOP*samplerate = n of audio frames between successive ffts
N_MELS = 40 # number of mel bins
WINDOW = 'hann' # each frame of audio is windowed by a window function (its length can also be
# determined and is then padded with zeros to match n_fft. we use window_length = length of fft

In [6]:
def data_from_file(filename):
    """
    Function that reads audio data audio
    saved in txt format
    (Helper function for read_wav_txt)

    Parameters
    ----------
    data: String
          path to txt file
          
    Returns
    -------
    data : 1D np.array
           Raw audio data (Amplitude)
    """
    data = pd.read_csv(filename, skiprows = 1,header = None)
    return np.asarray(data.iloc[:,0])

def sr_from_file(filename):
    """
    Function that reads audio sr from audio
    saved in txt format
    (Helper function for read_wav_txt)

    Parameters
    ----------
    data: String
          path to txt file
          
    Returns
    -------   
    sr: numeric (Integer)
        Samplerate (in Hz)
    """
    f = open(filename)
    line = f.readline()
    sr = line.split(':')[1]
    return int(sr)

def read_wav_txt(filename):    
    """
    Function that reads audio data and sr from audio
    saved in txt format

    Parameters
    ----------
    data: String
          path to txt file
          
    Returns
    -------
    data : 1D np.array
           Raw audio data (Amplitude)
           
    sr: numeric (Integer)
        Samplerate (in Hz)
    """
    data = "NA"
    sr = "NA"
    
    if os.path.exists(filename):
        data = data_from_file(filename)
        sr = sr_from_file(filename)
    else:
        print("No such file or directory: ", filename)
        data = np.asarray([0])
        sr = 0
    return data, sr

def generate_mel_spectrogram(data, rate, n_mels, window, fft_win , fft_hop):
    
    """
    Function that generates mel spectrogram from audio data using librosa functions

    Parameters
    ----------
    data: 1D numpy array (float)
          Audio data
    rate: numeric(integer)
          samplerate in Hz
    n_mels: numeric (integer)
            number of mel bands
    window: string
            spectrogram window generation type ('hann'...)
    fft_win: numeric (float)
             window length in s
    fft_hop: numeric (float)
             hop between window start in s 

    Returns
    -------
    result : 2D np.array
             Mel-transformed spectrogram

    Example
    -------
    >>> 
    
    """
    n_fft  = int(fft_win * rate) 
    hop_length = int(fft_hop * rate) 
        
    s = librosa.feature.melspectrogram(y = data ,
                                       sr = rate, 
                                       n_mels = n_mels , 
                                       fmax = 4000, 
                                       n_fft = n_fft,
                                       hop_length = hop_length, 
                                       window = window, 
                                       win_length = n_fft)

    spectro = librosa.power_to_db(s, ref=np.max)

    return spectro

# Read in data

In [12]:
# Read in labelfile
labelfile = pd.read_csv(LABELFILE, sep="\t")
# remove zero duration calls
labelfile = labelfile.loc[labelfile.duration_s>0,:]
# remove non-calls
labelfile = labelfile.loc[labelfile.isCall==1,:]

# Generate audio and samplerate column

In [ ]:
audios_we_need = [os.path.join(os.path.sep, TXT_PATH, x+'.txt') for x in labelfile.callID_new]
raw_audio,samplerate_hz = map(list,zip(*[read_wav_txt(x) for x in audios_we_need]))

labelfile['raw_audio'] = raw_audio
labelfile['samplerate_hz'] = samplerate_hz

# Generate spectrogram columns

In [ ]:
spectrograms = labelfile.apply(lambda row: generate_mel_spectrogram(row['raw_audio'],
                                                                    row['samplerate_hz'],
                                                                    N_MELS,
                                                                    WINDOW,
                                                                    FFT_WIN,
                                                                    FFT_HOP), 
                               axis=1)


labelfile['spectrograms'] = spectrograms

denoised = [(spectrogram - np.median(spectrogram, axis=0)) for spectrogram in labelfile['spectrograms']]
labelfile['denoised_spectrograms'] = denoised

# Save full labelfile

In [4]:
#labelfile.shape

(54386, 34)

In [ ]:
labelfile.to_pickle(os.path.join(os.path.sep, HOME, "full_labelfile.pkl"))